# 2. Reranker 최적화
### 절차
* 원본 데이터셋 기준, first-stage retriever 단계에서 최고 성능의 조합에 대해 Reranker 최적화 진행
* 파라미터 조합
    * k: 15
    * alpha: 20
    * dense_type: mrr
    * morphological_analyzer: bm25_kiwi
    * fetch_k: 2.5
    * lambda_mult: 0.7


### Reranker 종류
* cohere: reranker 3.5: $2.00/1k Searchs(https://docs.cohere.com/docs/rerank-overview)
* flash reranker: lightweight pairwise rerankers(https://github.com/PrithivirajDamodaran/FlashRank)
* jina reranker: jina-reranker-v2-base-multilingual (https://huggingface.co/jinaai/jina-reranker-v2-base-multilingual)

In [ ]:
import cohere
from flashrank import Ranker, RerankRequest

In [ ]:
dataset.head(2)

,user_input,reference_contexts,reference,synthesizer_name,optimal_retrieve
0,역도 대회에서 엄지(thumb) 사용에 관한 규정은 무엇인가요?,"['3) 벨트의 최대 넓이는 12cm를 넘지 말아야 한다. 4) 붕대, 반창고 혹은...",역도 대회에서 붕대(bandage) 혹은 석고(cast)는 손가락(finger) 혹...,single_hop_specifc_query_synthesizer,['평소 건강했던 선수도 갑자기 신체의 조정 기구에 이상이 있으면 피부색이나 얼\n...
1,바벨의 무게를 들어올리기 위해 필요한 회전력은 얼마인가요?,"['인상동작과 시작자세 → 끌기(first pull)자세 → 몸통 펴기(잡아채기, ...",바벨이 150kg이고 바벨의 수직축과 엉덩이 관절까지의 거리가 60cm라면 바벨을 ...,single_hop_specifc_query_synthesizer,['그\n러나 엉덩이를 앞으로 넣는 자세를 취하여 이 거리를 50cm로 줄인다면 약...


In [ ]:
co = cohere.ClientV2()
query = dataset.iloc[0, 0]
docs = dataset.iloc[0, -1]

results = co.rerank(model='rerank-v3.5', query=query, documents=docs)

In [ ]:
results.results

[V2RerankResponseResultsItem(document=None, index=3, relevance_score=0.23717858),
 V2RerankResponseResultsItem(document=None, index=2, relevance_score=0.18431228),
 V2RerankResponseResultsItem(document=None, index=8, relevance_score=0.14607534),
 V2RerankResponseResultsItem(document=None, index=7, relevance_score=0.11839552),
 V2RerankResponseResultsItem(document=None, index=5, relevance_score=0.114774935),
 V2RerankResponseResultsItem(document=None, index=6, relevance_score=0.09654403),
 V2RerankResponseResultsItem(document=None, index=10, relevance_score=0.0954255),
 V2RerankResponseResultsItem(document=None, index=13, relevance_score=0.09282766),
 V2RerankResponseResultsItem(document=None, index=1, relevance_score=0.08735358),
 V2RerankResponseResultsItem(document=None, index=12, relevance_score=0.06917),
 V2RerankResponseResultsItem(document=None, index=4, relevance_score=0.065563075),
 V2RerankResponseResultsItem(document=None, index=9, relevance_score=0.05382461),
 V2RerankRespon

In [ ]:
ranker = Ranker(model_name="ms-marco-MultiBERT-L-12", cache_dir="/opt")
passages = [{'id': i, 'text': doc} for i, doc in enumerate(docs)]
rerankrequest = RerankRequest(query=query, passages=passages)
results = ranker.rerank(rerankrequest)
results = [result['text'] for result in results]

INFO:flashrank.Ranker:Downloading ms-marco-MultiBERT-L-12...
ms-marco-MultiBERT-L-12.zip: 100%|██████████| 98.7M/98.7M [03:10<00:00, 543kiB/s] 


In [ ]:
class RerankerPipeline:
    def __init__(self):
        self.flash_ranker = Ranker(model_name="ms-marco-MiniLM-L-12-v2")        
        self.cohere_client = cohere.Client(os.getenv('COHERE_API_KEY'))
        
        # Jina API 설정
        self.jina_headers = {
            'Authorization': f'Bearer {os.getenv('JINA_API_KEY')}'
        }

    def rerank_with_flashrank(self, query: str, passages: List[str]) -> List[Dict]:
        """FlashRank lightweight pairwise reranker를 사용한 reranking"""
        # FlashRank의 올바른 입력 형식으로 변환
        passages_with_id = [{"id": i, "text": text} for i, text in enumerate(passages)]
        results = self.flash_ranker.rerank(
            query=query,
            passages=passages_with_id
        )
        return results

    def rerank_with_cohere(self, query: str, passages: List[str]) -> List[Dict]:
        """Cohere를 사용한 reranking"""
        results = self.cohere_client.rerank(
            query=query,
            documents=passages,
            model='rerank-english-v2.0',
            top_n=len(passages)
        )
        return results

    def rerank_with_jina(self, query: str, passages: List[str]) -> List[Dict]:
        """Jina를 사용한 reranking"""
        url = "https://api.jina.ai/v1/rerank"
        payload = {
            "query": query,
            "documents": passages
        }
        response = requests.post(url, headers=self.jina_headers, json=payload)
        return response.json()

In [ ]:
def process_dataframe(reranker, df: pd.DataFrame) -> pd.DataFrame:
    """데이터프레임의 각 행에 대해 reranking 수행"""
    reranker = RerankerPipeline()
    
    # 데이터프레임 복사본 생성
    result_df = df.copy()
    
    # 결과를 저장할 새로운 컬럼들
    result_df.loc[:, 'flashrank_results'] = None
    result_df.loc[:, 'cohere_results'] = None
    result_df.loc[:, 'jina_results'] = None
    
    for idx, row in result_df.iterrows():
        query = row['user_input']
        passages = row['optimal_retrieve']
        
        try:
            # FlashRank reranking
            flashrank_results = reranker.rerank_with_flashrank(query, passages)
            result_df.loc[idx, 'flashrank_results'] = flashrank_results
            
            # Cohere reranking
            cohere_results = reranker.rerank_with_cohere(query, passages)
            result_df.loc[idx, 'cohere_results'] = cohere_results
            
            # Jina reranking
            jina_results = reranker.rerank_with_jina(query, passages)
            result_df.loc[idx, 'jina_results'] = jina_results
            
            # API 호출 제한을 위한 딜레이
            time.sleep(1)
            
        except Exception as e:
            print(f"Error processing row {idx}: {str(e)}")
            continue
    
    return result_df

In [ ]:
def get_optimal_retrieval_results(df, optimal_params):
    """
    최적 파라미터 조합으로 검색 결과를 생성하는 함수
    """
    results = []
    
    for _, row in df.iterrows():
        # Dense 결과 처리
        if optimal_params['dense_type'] == 'mrr':
            fetch_k = int(optimal_params['fetch_k'] * optimal_params['k'])
            lambda_mult = optimal_params['lambda_mult']
            
            dense_results = apply_mrr_ranking(
                [(doc.__dict__['page_content'], score) for doc, score in row['precompute_dense'][:fetch_k]], 
                lambda_mult
            )
        else:
            dense_results = [doc.__dict__['page_content'] for doc, score in row['precompute_dense']]
        
        # Sparse 결과 처리
        if optimal_params['morphological_analyzer'] == 'bm25_kiwi':
            sparse_results = row['precompute_sparse_bm25_kiwi']
        elif optimal_params['morphological_analyzer'] == 'bm25':
            sparse_results = row['precompute_sparse_bm25']
        else:  # bm25_kiwi_pos
            sparse_results = row['precompute_sparse_bm25_kiwi_pos']
        
        combined_results = combine_hybrid_results(
            [dense_results], 
            [sparse_results], 
            optimal_params['alpha'],
            optimal_params['k']
        )
        
        results.append(combined_results[0])  # combine_hybrid_results는 리스트의 리스트를 반환하므로 첫 번째 요소만 사용
    
    return results

optimal_params = {
    'k': 15,
    'alpha': 40,
    'dense_type': 'mrr',
    'morphological_analyzer': 'bm25_kiwi',
    'fetch_k': 2.5,
    'lambda_mult': 0.7
}

In [ ]:
# optimal_results = get_optimal_retrieval_results(origin_dataset, optimal_params)
# dataset = origin_dataset.iloc[:, :-4].copy()
# dataset['optimal_retrieve'] = optimal_results
# dataset.to_csv('../data/document/역도/optimal_retrieve.csv', index=False)

# tmp = custom_dataset.drop(columns=['reference_contexts_section'])
# optimal_results = get_optimal_retrieval_results(tmp, optimal_params)
# dataset = tmp.iloc[:, :-4].copy()
# dataset['optimal_retrieve'] = optimal_results
# dataset.to_csv('../data/document/역도/custom_optimal_retrieve.csv', index=False)

In [ ]:
dataset = pd.read_csv('../data/document/역도/optimal_retrieve.csv')
custom_dataset = pd.read_csv('../data/document/역도/custom_optimal_retrieve.csv')

dataset['reference_contexts'] = dataset['reference_contexts'].apply(lambda x : eval(x))
dataset['optimal_retrieve'] = dataset['optimal_retrieve'].apply(lambda x : eval(x))

custom_dataset['reference_contexts'] = custom_dataset['reference_contexts'].apply(lambda x : eval(x))
custom_dataset['optimal_retrieve'] = custom_dataset['optimal_retrieve'].apply(lambda x : eval(x))

In [ ]:
tmp

,user_input,reference_contexts,reference,synthesizer_name,optimal_retrieve,flashrank_results,cohere_results,jina_results
0,역도 대회에서 엄지(thumb) 사용에 관한 규정은 무엇인가요?,"['3) 벨트의 최대 넓이는 12cm를 넘지 말아야 한다. 4) 붕대, 반창고 혹은...",역도 대회에서 붕대(bandage) 혹은 석고(cast)는 손가락(finger) 혹...,single_hop_specifc_query_synthesizer,['평소 건강했던 선수도 갑자기 신체의 조정 기구에 이상이 있으면 피부색이나 얼\n...,None,None,None
1,바벨의 무게를 들어올리기 위해 필요한 회전력은 얼마인가요?,"['인상동작과 시작자세 → 끌기(first pull)자세 → 몸통 펴기(잡아채기, ...",바벨이 150kg이고 바벨의 수직축과 엉덩이 관절까지의 거리가 60cm라면 바벨을 ...,single_hop_specifc_query_synthesizer,['그\n러나 엉덩이를 앞으로 넣는 자세를 취하여 이 거리를 50cm로 줄인다면 약...,None,None,None
2,초급자에게 권장되는 훈련 빈도는 얼마인가요?,"[""초급자는 일주일에 3번 훈련하는 것이 바람직하며, 기록의 향상보다는 기술의 완\...",초급자는 일주일에 3번 훈련하는 것이 바람직하다.,single_hop_specifc_query_synthesizer,"[""초급자는 일주일에 3번 훈련하는 것이 바람직하며, 기록의 향상보다는 기술의 완\...",None,None,None
3,"이완이란 무엇이며, 어떻게 심리기술 훈련에 활용될 수 있나요?","['시합에서 발생하는 심리적, 전술적 위기를 막기 위해 심리적 위기를 유발하는\n심...","이완은 신체적인 긴장이나 심리적인 긴장을 완화시켜 신체적, 심리적으로 안정된 상태에...",single_hop_specifc_query_synthesizer,"['시합에서 발생하는 심리적, 전술적 위기를 막기 위해 심리적 위기를 유발하는\n심...",None,None,None
4,상체 훈련은 역도에서 어떤 역할을 하나요?,['이미지는 역도 경기의 Top Snatch 훈련 방법을 단계별로 보여준다. 첫 번...,"상체는 역도에서 중요한 역할을 하며, 선수는 바벨을 머리 위로 들어 올릴 때 상체를...",single_hop_specifc_query_synthesizer,['아래에 제시한 체력의 요소들은 현장에서 일반적으로 행하고 있고 역도기초체력\n을...,None,None,None


# Generator 최적화
* 평가지표
    * meteor
    * rouge
    * sem_score
    * bert_score
* 파라미터
    * temperature: [0, 0.2, 0.4, 0.6, 0.8, 1.0]
    * max_tokens: [512, 1024]
    * llm: [gpt-3.5-turbo, gpt-4-turbo]